In [ ]:
!pip install fastapi uvicorn pyngrok  pandas
!pip install python-multipart
!pip install scikit-learn==1.3.0
!pip install imbalanced-learn==0.11.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, File, UploadFile
import pandas as pd
from typing import List
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(debug=True)

@app.get("/")
async def read_root():
    logger.info("Root endpoint accessed")
    return {"message": "Welcome to the Machine Downtime Prediction API!"}

@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    logger.info(f"Upload endpoint accessed with file: {file.filename}")
    try:
        global dataset
        contents = await file.read()
        with open('temp.csv', 'wb') as f:
            f.write(contents)
        dataset = pd.read_csv('temp.csv')

        return {
            "message": f"File '{file.filename}' uploaded successfully",
            "columns": dataset.columns.tolist()
        }
    except Exception as e:
        logger.error(f"Error processing upload: {str(e)}")
        return {"error": str(e)}

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Generate synthetic data
num_records = 1000
data = {
    "Machine_ID": np.random.randint(1, 21, num_records),  # 20 unique machine IDs
    "Temperature": np.random.uniform(50, 100, num_records),  # Random temperatures between 50 and 100
    "Run_Time": np.random.uniform(1, 1000, num_records),  # Random run times between 1 and 1000 hours
    "Downtime_Flag": np.random.choice([0, 1], num_records, p=[0.7, 0.3])  # 70% No downtime, 30% downtime
}

df = pd.DataFrame(data)

file_name = "machine_downtime.csv"
df.to_csv(file_name, index=False)

print(f"Synthetic data saved as '{file_name}'")
print(df.head())


Synthetic data saved as 'machine_downtime.csv'
   Machine_ID  Temperature    Run_Time  Downtime_Flag
0           7    73.285006  895.918783              1
1          20    74.041849  322.893288              0
2          15    95.922736  126.108337              0
3          11    79.353411  474.861774              0
4           8    51.642334  114.842678              1


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading
import time

nest_asyncio.apply()

ngrok.kill()

ngrok.set_auth_token("<YOUR_NGROK_TOKEN>")
ngrok_tunnel = ngrok.connect(8000)
public_url = ngrok_tunnel.public_url
print('Public URL:', public_url)

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(5)

Public URL: https://229f-34-139-187-100.ngrok-free.app


INFO:     Started server process [575]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     223.233.86.126:0 - "GET / HTTP/1.1" 200 OK


In [ ]:
import requests

url = "<YOUR_NGROK_PUBLIC_URL>/upload/"  # Replace <YOUR_NGROK_PUBLIC_URL> with your actual ngrok URL
files = {"file": open("machine_downtime.csv", "rb")}
response = requests.post(url, files=files)

print(response.json())


INFO:     34.139.187.100:0 - "POST /upload/ HTTP/1.1" 200 OK
{'message': "File 'machine_downtime.csv' uploaded successfully", 'columns': ['Machine_ID', 'Temperature', 'Run_Time', 'Downtime_Flag']}


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
import joblib
from fastapi import HTTPException

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
import numpy as np

scaler = StandardScaler()
@app.post("/train")
async def train_model():
    try:
        global dataset, model, scaler

        if dataset is None:
            return {"error": "No dataset uploaded. Please upload dataset first."}

        X = dataset[["Temperature", "Run_Time"]]
        y = dataset["Downtime_Flag"]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(
            X_train_scaled, y_train
        )

        model = LogisticRegression(
            C=1.0,
            penalty='l2',
            solver='lbfgs',
            max_iter=1000,
            random_state=42
        )

        model.fit(X_train_resampled, y_train_resampled)

        y_pred = model.predict(X_test_scaled)

        metrics = {
            "accuracy": np.round(accuracy_score(y_test, y_pred), 3),
            "f1_score": np.round(f1_score(y_test, y_pred), 3),
            "precision": np.round(precision_score(y_test, y_pred), 3),
            "recall": np.round(recall_score(y_test, y_pred), 3)
        }

        return {
            "message": "Model trained successfully",
            **metrics
        }

    except Exception as e:
        logger.error(f"Error training model: {str(e)}")
        return {"error": str(e)}

In [ ]:
import requests

# Replace <YOUR_NGROK_PUBLIC_URL> with your updated ngrok public URL
url = "<YOUR_NGROK_PUBLIC_URL>/train"

response = requests.post(url)

print(response.json())


INFO:     34.139.187.100:0 - "POST /train HTTP/1.1" 200 OK
{'message': 'Model trained successfully', 'accuracy': 0.525, 'f1_score': 0.424, 'precision': 0.343, 'recall': 0.556}


In [ ]:
@app.post("/predict")
async def predict_downtime(data: dict):
    try:

        required_fields = ["Temperature", "Run_Time"]
        if not all(field in data for field in required_fields):
            raise HTTPException(
                status_code=400,
                detail=f"Missing required fields. Please provide: {required_fields}"
            )

        input_data = pd.DataFrame([data])
        input_data = input_data[required_fields]  #

        input_scaled = scaler.transform(input_data)

        prediction = model.predict(input_scaled)[0]
        probability = model.predict_proba(input_scaled)[0][1]

        return {
            "Downtime": "Yes" if prediction == 1 else "No",
            "Confidence": round(float(probability), 3)
        }

    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
test_data = {
    "Temperature": 75.5,
    "Run_Time": 500
}

response = requests.post(
    "<YOUR_NGROK_PUBLIC_URL>/predict",
    json=test_data
)
print(response.json())

INFO:     34.139.187.100:0 - "POST /predict HTTP/1.1" 200 OK
{'Downtime': 'Yes', 'Confidence': 0.5}
